# 导入工具包

In [1]:
# opencv-python
import cv2
import numpy as np
# mediapipe人工智能工具包
import mediapipe as mp
# 进度条库
from tqdm import tqdm
# 时间库
import time

# 导入模型

In [2]:
# 导入三维人脸关键点检测模型
mp_face_mesh = mp.solutions.face_mesh
model = mp_face_mesh.FaceMesh(   
        static_image_mode=False,      # 是静态图片还是连续视频帧，摄像头画面为连续视频帧，此处选False
        refine_landmarks=True,       # 使用注意力机制Attention Mesh Model，对嘴唇、眼睛、瞳孔周围的关键点精细定位
        max_num_faces=5,              # 最多检测几张脸
        min_detection_confidence=0.5, # 置信度阈值
        min_tracking_confidence=0.5,  # 追踪阈值
)

# 导入可视化绘图函数
mp_drawing = mp.solutions.drawing_utils 
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1, color=[66,77,229])

# 处理单帧的函数

In [3]:
# 处理帧函数
def process_frame(img):
    
    # 记录该帧开始处理的时间
    start_time = time.time()

    # 获取图像宽高
    h,w = img.shape[0], img.shape[1]
    # BGR转RGB
    img_RGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # 将RGB图像输入模型，获取预测结果
    results = model.process(img_RGB)

    # 文字大小
    scaler = 1

    if results.multi_face_landmarks: # 如果检测出人脸

        # 获取相关关键点坐标
        # 脸轮廓最左点
        FL = results.multi_face_landmarks[0].landmark[234]; FL_X, FL_Y = int(FL.x * w), int(FL.y * h); FL_Color = (0,0,255)
        img = cv2.circle(img,(FL_X, FL_Y), 5, FL_Color, -1)
        # 脸上边缘
        FT = results.multi_face_landmarks[0].landmark[10]; FT_X, FT_Y = int(FT.x * w), int(FT.y * h); FT_Color = (31,41,81)
        # img = cv2.circle(img,(FT_X, FT_Y), 5, FT_Color, -1)
        # 脸下边缘
        FB = results.multi_face_landmarks[0].landmark[152]; FB_X, FB_Y = int(FB.x * w), int(FB.y * h); FB_Color = (31,41,81)
        # img = cv2.circle(img,(FB_X, FB_Y), 5, FB_Color, -1)
        # 脸轮廓最右点
        FR = results.multi_face_landmarks[0].landmark[454]; FR_X, FR_Y = int(FR.x * w), int(FR.y * h); FR_Color = (0,255,0)
        img = cv2.circle(img,(FR_X, FR_Y), 5, FR_Color, -1)
        # 左边眼睛左眼角
        ELL = results.multi_face_landmarks[0].landmark[33]; ELL_X, ELL_Y = int(ELL.x * w), int(ELL.y * h); ELL_Color = (255,0,0)
        img = cv2.circle(img,(ELL_X, ELL_Y), 5, ELL_Color, -1)
        # 左边眼睛右眼角
        ELR = results.multi_face_landmarks[0].landmark[133]; ELR_X, ELR_Y = int(ELR.x * w), int(ELR.y * h); ELR_Color = (0,255,255)
        img = cv2.circle(img,(ELR_X, ELR_Y), 5, ELR_Color, -1)
        # 右边眼睛左眼角
        ERL = results.multi_face_landmarks[0].landmark[362]; ERL_X, ERL_Y = int(ERL.x * w), int(ERL.y * h); ERL_Color = (223,155,6)
        img = cv2.circle(img,(ERL_X, ERL_Y), 5, ERL_Color, -1)
        # 右边眼睛右眼角
        ERR = results.multi_face_landmarks[0].landmark[263]; ERR_X, ERR_Y = int(ERR.x * w), int(ERR.y * h); ERR_Color = (151,57,224)
        img = cv2.circle(img,(ERR_X, ERR_Y), 5, ERR_Color, -1)

        # 计算“五眼指标”
        # 从左往右六个点的横坐标
        Six_X = np.array([FL_X, ELL_X, ELR_X, ERL_X, ERR_X, FR_X])
        # 从最左到最右的距离
        Left_Right = FR_X - FL_X
        # 从左往右六个点间隔的五个距离，并归一化
        Five_Distance = 100 * np.diff(Six_X) / Left_Right
        # 两眼宽度的平均值
        Eye_Width_Mean = np.mean([Five_Distance[1], Five_Distance[3]])
        # 五个距离 与 两眼宽度均值 的差
        Five_Eye_Diff = Five_Distance - Eye_Width_Mean
        # 求L2范数，作为颜值的“五眼”评价指标
        Five_Eye_Metrics = np.linalg.norm(Five_Eye_Diff)
        # 画直线
        cv2.line(img,(FL_X, FT_Y),(FL_X, FB_Y), FL_Color, 2)
        cv2.line(img,(ELL_X, FT_Y),(ELL_X, FB_Y), ELL_Color, 2)
        cv2.line(img,(ELR_X, FT_Y),(ELR_X, FB_Y), ELR_Color, 2)
        cv2.line(img,(ERL_X, FT_Y),(ERL_X, FB_Y), ERL_Color, 2)
        cv2.line(img,(ERR_X, FT_Y),(ERR_X, FB_Y), ERR_Color, 2)
        cv2.line(img,(FR_X, FT_Y),(FR_X, FB_Y), FR_Color, 2)
        cv2.line(img,(FL_X, FT_Y),(FR_X, FT_Y), FT_Color, 2)
        cv2.line(img,(FL_X, FB_Y),(FR_X, FB_Y), FB_Color, 2)

        scaler = 1
        img = cv2.putText(img, 'Five Eye Metrics {:.2f}'.format(Five_Eye_Metrics), (25 * scaler, 100 * scaler), cv2.FONT_HERSHEY_SIMPLEX, 1.25 * scaler, (255, 0, 255), 2 * scaler)
        img = cv2.putText(img, 'Distance 1 {:.2f}'.format(Five_Eye_Diff[0]), (25 * scaler, 150 * scaler), cv2.FONT_HERSHEY_SIMPLEX, 1.25 * scaler, (255, 0, 255), 2 * scaler)
        img = cv2.putText(img, 'Distance 2 {:.2f}'.format(Five_Eye_Diff[2]), (25 * scaler, 200 * scaler), cv2.FONT_HERSHEY_SIMPLEX, 1.25 * scaler, (255, 0, 255), 2 * scaler)
        img = cv2.putText(img, 'Distance 3 {:.2f}'.format(Five_Eye_Diff[4]), (25 * scaler, 250 * scaler), cv2.FONT_HERSHEY_SIMPLEX, 1.25 * scaler, (255, 0, 255), 2 * scaler)

    else:
        img = cv2.putText(img, 'No Face Detected', (25 * scaler, 50 * scaler), cv2.FONT_HERSHEY_SIMPLEX, 1.25 * scaler, (255, 0, 255), 2 * scaler)
    
    # 记录该帧处理完毕的时间
    end_time = time.time()
    # 计算每秒处理图像帧数FPS
    FPS = 1/(end_time - start_time)
    img = cv2.putText(img, 'FPS  '+str(int(FPS)), (25 * scaler, 50 * scaler), cv2.FONT_HERSHEY_SIMPLEX, 1.25 * scaler, (255, 0, 255), 2 * scaler)
    return img

# 调用摄像头获取每帧（模板）

In [4]:
# 调用摄像头逐帧实时处理模板
# 不需修改任何代码，只需修改process_frame函数即可

# 导入opencv-python
import cv2
import time

# 获取摄像头，传入0表示获取系统默认摄像头
cap = cv2.VideoCapture(1)

# 打开cap
cap.open(0)

# 无限循环，直到break被触发
while cap.isOpened():
    # 获取画面
    success, frame = cap.read()
    if not success:
        print('Error')
        break
    start_time = time.time()
    
    ## !!!处理帧函数
    frame = process_frame(frame)
    
    # 展示处理后的三通道图像
    cv2.imshow('my_window',frame)

    if cv2.waitKey(1) in [ord('q'),27]: # 按键盘上的q或esc退出（在英文输入法下）
        break
    
# 关闭摄像头
cap.release()

# 关闭图像窗口
cv2.destroyAllWindows()

如果遇到Error，需检查其它notebook是否占用了摄像头，需在其它notebook中restart kernel

# 视频逐帧处理（模板）

In [5]:
# 视频逐帧处理代码模板
# 不需修改任何代码，只需定义process_frame函数即可

def generate_video(input_path='./videos/three-hands.mp4'):
    filehead = input_path.split('/')[-1]
    output_path = "out-" + filehead
    
    print('视频开始处理',input_path)
    
    # 获取视频总帧数
    cap = cv2.VideoCapture(input_path)
    frame_count = 0
    while(cap.isOpened()):
        success, frame = cap.read()
        frame_count += 1
        if not success:
            break
    cap.release()
    print('视频总帧数为',frame_count)
    
    # cv2.namedWindow('Crack Detection and Measurement Video Processing')
    cap = cv2.VideoCapture(input_path)
    frame_size = (cap.get(cv2.CAP_PROP_FRAME_WIDTH), cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # fourcc = int(cap.get(cv2.CAP_PROP_FOURCC))
    # fourcc = cv2.VideoWriter_fourcc(*'XVID')
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = cap.get(cv2.CAP_PROP_FPS)

    out = cv2.VideoWriter(output_path, fourcc, fps, (int(frame_size[0]), int(frame_size[1])))
    
    # 进度条绑定视频总帧数
    with tqdm(total=frame_count-1) as pbar:
        try:
            while(cap.isOpened()):
                success, frame = cap.read()
                if not success:
                    break

                # 处理帧
                # frame_path = './temp_frame.png'
                # cv2.imwrite(frame_path, frame)
                try:
                    frame = process_frame(frame)
                except:
                    print('error')
                    pass
                
                if success == True:
                    # cv2.imshow('Video Processing', frame)
                    out.write(frame)

                    # 进度条更新一帧
                    pbar.update(1)

                # if cv2.waitKey(1) & 0xFF == ord('q'):
                    # break
        except:
            print('中途中断')
            pass

    cv2.destroyAllWindows()
    out.release()
    cap.release()
    print('视频已保存', output_path)

In [ ]:
generate_video(input_path='videos/single_person.mp4')